In [57]:
# standard
import numpy as np
import pandas as pd

# web scraping stuff
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains

import requests

# Misc
import cv2 # for image processing
import mss # for screenshots and stuff
import regex # regex

# # Inbuilt libraries
import copy
import time
import json
import os

FOR WINDOWS:

cd C:\Program Files (x86)\Google\Chrome\Application

chrome.exe --remote-debugging-port=9250 --user-data-dir="c:\users\Charl\chromedriver"

url = "https://lichess.org"
ser = Service("C:\\Users\\Charl\\chromedriver.exe")
opts = Options()
opts.add_experimental_option('debuggerAddress', 'localhost:9250')
driver = webdriver.Chrome(service= ser, options=opts)
driver.get(url)


In [58]:
# Also need stockfish, follow instructions here
# https://pypi.org/project/stockfish/
# For mac os do brew install, then just stockfish()
# for windows you need to specifiy the binary path

In [59]:
# Stockfish
from stockfish import Stockfish
stockfish = Stockfish()

# path="stockfish//stockfish_14.exe")

In [60]:
# you need to download chrome driver from here, and point to where you saved it
# https://chromedriver.chromium.org/downloads
# just save it in the folder and the below script should run

In [61]:
ser = Service(executable_path= os.getcwd() + "/chromedriver") 
opts = Options()
opts.add_argument("user-data-dir=selenium") 
driver = webdriver.Chrome(service= ser, options=opts)
driver.get('https://lichess.org')
time.sleep(5)

In [62]:
def find_square_size():
    '''Gets the width of a squard in pixels, for use later
    returns: float or int of square width'''
    height_and_width = driver.find_element(By.XPATH, "//cg-container").get_attribute('style')
    board_size = float(height_and_width.split(' ')[1].split('px')[0])
    
    # If board size is divisble by 8, then return integer, else return float.
    # need to test with float sizez
    
    square_size = board_size / 8
    
    
    return int(square_size) if board_size % 8 == 0 else float(square_size) # else what?

In [63]:
def find_color():
    '''Finds the color we are and returns it (white / black)'''
    find_piece_example = "//cg-board/piece[contains (@style, '(0px, 0px')]"

    find_color_white = "//div[@class='round__app__board main-board']/div [contains (@class, 'white')]"
    find_color_black = "//div[@class='round__app__board main-board']/div [contains (@class, 'black')]"

    try:
        driver.find_element(By.XPATH, find_color_white)
        color = 'white'
    except Exception:
        try:
            driver.find_element(By.XPATH, find_color_black)
            color ='black'
        except Exception:
            return False
            

    return color

In [64]:
def get_move_dicts(color):
    '''Generates square: pixel value mappings
    
    args: 
        color (string): color we are (white / black)
        
    returns:
        dict of mappings + reversed dict'''
    if color == 'white':
        move_dict = {}
        for letter_index, letter in enumerate(['a', 'b', 'c', 'd', 'e', 'f', 'g','h']):
            for number_index, number in enumerate([str(i) for i in range(8, 0, -1)]):

                row_px = letter_index * square_size
                col_px = number_index * square_size
                move_dict[letter + number] = f'({row_px:.0f}px, {col_px:.0f}px)'

    elif color == 'black':
        move_dict = {}
        for letter_index, letter in enumerate(['h', 'g', 'f', 'e', 'd', 'c', 'b' ,'a']):
            for number_index, number in enumerate([str(i) for i in range(1, 9)]):

                row_px = letter_index * square_size
                col_px = number_index * square_size
                move_dict[letter + number] = f'({row_px:.0f}px, {col_px:.0f}px)'

    move_dict_reversed = {value: key for key, value in move_dict.items()}
    
    return move_dict, move_dict_reversed

In [65]:
def make_move(move, move_dict):
    '''Executes the desired move on lichess
    args:
        [move]: string of 4 characters e.g a2a4'''
    # Chain of events. add events then perform all with action.perform()
    action = ActionChains(driver=driver) 
    
    start_square = move[:2]
    end_square = move[2:4]
    
    # Find pixel values
    start_pixel_value = move_dict[start_square]
    end_pixel_values = move_dict[end_square]
    
    ele = driver.find_element(By.XPATH, 
                              "//cg-container/cg-board/" + f"piece[contains (@style, '{start_pixel_value}')]")

    # Click and hold the piece we want to move
    action.click(ele)
    action.perform()

    # Wait for the move markers to show up, not sure what time necessary here (maybe 0)
    time.sleep(1)

    # Below will need to change to the pixel location
    ele2 = driver.find_element(By.XPATH,
                               "//cg-container/cg-board/" + f"square[contains (@style, '{end_pixel_values}')]")

    # Move the mouse to the end square and drop
    #action.click_and_hold(ele)
    action.click(ele2)
    time.sleep(0.2)

    action.perform()

In [66]:
def check_if_my_turn():
    '''Checks if we are to move
    
    First check if we have a clock, and it's running.
    Else check if there is a 'your turn' element
    '''
    
    try:
        ele = driver.find_element(
            By.XPATH,
            "//div[contains (@class, 'rclock') and contains(@class, 'rclock-bottom')]")
        return bool(ele.get_attribute('class'))
    except Exception:
        return False

In [67]:
def get_previous_move():
    """Get the last move made (start & end square) if exists, else return None"""
    try:
        last_move_end = driver.find_element(
            By.XPATH,
            base_expression + "square[contains (@class, 'last-move')][1]"
        ).get_attribute('style').split('translate')[1].split(';')[0]
        
        last_move_start = driver.find_element(
            By.XPATH, 
            base_expression + "square[contains (@class, 'last-move')][2]"
        ).get_attribute('style').split('translate')[1].split(';')[0]
        
        # Translating to string value now (might be unnecessary)
        last_move_end = move_dict_reversed[last_move_end]
        last_move_start = move_dict_reversed[last_move_start]
        
        return ''.join([last_move_start, last_move_end])
    except Exception:
        return None
        


Troubleshooting below ####

In [70]:
previous_moves = []
square_size = find_square_size()
color = find_color()
if not color:
    raise Exception('No color found! Is the board loaded?')
print('Our color is ', color)
move_dict, move_dict_reversed = get_move_dicts(color)

while True:
    
    # Check board is still loaded:
    if not find_color():
        print('No color found! Is the board loaded?')
        break
        
    #my_turn = check_if_my_turn()
    # Update turn by checking for previous moves?
    if color == 'white':
        my_turn = ((len(previous_moves) % 2) == 0)
    if color == 'black':
        my_turn = ((len(previous_moves) % 2) == 1)
        
    if not my_turn:
        
        last_detected_move = get_previous_move()
        
        # print('previous moves', previous_moves)
        # print('last detected move', last_detected_move)
        
        if not last_detected_move is None: # Incase we are black and no moves yet
            
            # if new move then add to the list!
            
            
            
            if len(previous_moves) == 0:
                previous_moves.append(last_detected_move)
                
            
            elif last_detected_move != previous_moves[-1]:
                previous_moves.append(last_detected_move)
            
            # print('last move added', previous_moves[-1])
        

    if my_turn:
            
        while True:

            stockfish.set_position(previous_moves)
            while True:
                new_move = input('give me the next move plz')
                
                # If valid move then carry on
                if isinstance(new_move, str) & (len(new_move) == 4):
                    break 

            is_correct_move = stockfish.is_move_correct(new_move)
            if is_correct_move:
                make_move(new_move, move_dict)
                previous_moves.append(new_move)
                break
            else:
                print('wrong move')

Our color is  black


KeyboardInterrupt: 

In [ ]:
def run_checks(previous_moves=[]):
    
    print('my color', find_color())
    print('is my turn?', check_if_my_turn())
    print('previous moves:', previous_moves)
    print('square_size', find_square_size())
    print('last detected_move', get_previous_move())
    
try:
    run_checks(previous_moves)
except NameErorr:
    run_checks()

In [ ]:
# bug on pawn moving from c7 to f3, when it actually moved from c7 to c5